# War Modeling
## Money or Soldiers : What's more important

<img src="https://ipython-books.github.io/pages/chapter06_viz/04_d3_files/D3.png" width=500px>

**Group members:** 
<br>Jonathan Le Roux
<br>Pranav Agarwal
<br>Bhavya Jain
<br>Ethan Hecht

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import time
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import math


In [ ]:
class Territory:
    """
    Class that houses the information for each individual territory of a country
    
    Attributes:
    position - a list in the form longitude, latitude
    num_soldiers - a numeric value which has the number of soldiers a territory has in thousands
    
    
    Methods:
    add_soldiers - increases the number of soldiers and adds the new soldiers skill levels to the dictionary
    attack = fights oppositing territory given in the argument, updates the teritories army
             after war and returns the opposing territory with an updated army. 
    """
    def __init__(self,name, position, initial_population, mean_skill_level = 0.3, population_growth_rate = 0.01):
        
        self.population = initial_population
        self.position = position
        self.name = name
        self.money = 1
        
        self.deaths =0
        
        #below sets each soldier skill level randomly
        self.num_soldiers = int(0.005*initial_population)
        self.mean_skill_level = mean_skill_level
        self.prev_money = self.money
        self.skill_levels = np.abs(np.random.normal(mean_skill_level, 0.2, self.num_soldiers))
        self.population_growth_rate = population_growth_rate
               
    def get_position(self):
        return self.position
    
    def set_position(self, position):
        self.position = position
        
    def get_population(self):
        return self.population
    
    def get_num_soldiers(self):
        return len(self.skill_levels)
    
    def add_soldiers(self, mean_skill_level=0.2):
        number = 0
        if (len(self.skill_levels) < 0.005 * self.population):
            number = int((0.005 * self.population) - len(self.skill_levels))
            self.skill_levels = np.append(self.skill_levels, np.abs(np.random.normal(mean_skill_level, 0.2, number)))
        return number  
            
    def add_population(self):
        self.population = self.population_growth_rate * self.population
        
    def improve_skill(self):
        if self.money * 0.1>= 1000 :
            self.lose_money(percent = 0.1)
            #self.skill_levels = np.clip(np.abs(np.random.normal(self.mean_skill_level*1.01, 0.2, self.num_soldiers)),0,1)
            self.skill_levels = self.skill_levels*1.01
            
            
            
    def attack(self, opposing_territory):
        #changes skill level
        self.improve_skill()
        opposing_territory.improve_skill()
        #returns the winner
        attack = np.copy(self.skill_levels)
        defend = np.copy(opposing_territory.skill_levels)
        #randomizing the attack pattern of two armies
        np.random.shuffle(attack) 
        np.random.shuffle(defend)
        death_a, death_d = 0,0
       
        #run loop for minimum length steps till one of the army dies completely
        while attack.size!=0 and defend.size!=0:
            soldier1, soldier2 = attack[0], defend[0]
            probability = np.random.random()
            if soldier1/(soldier1 + soldier2) > probability:
                defend = np.delete(defend, 0)
                death_d += 1
            else:
                attack = np.delete(attack, 0)
                death_a += 1
                
        self.deaths += death_a
        opposing_territory.deaths += death_d
                
        self.skill_levels = attack
        opposing_territory.skill_levels = defend
        self.population = self.population - death_a
        opposing_territory.population = opposing_territory.population - death_d
        self.lose_money()
        opposing_territory.lose_money()
        if len(defend) == 0 :
            return self, opposing_territory
        else:
            return opposing_territory, self
        
    def lose_money(self, percent = 0.1):
        self.prev_money = np.copy(self.money)
        self.money -= self.money * percent
    
    def to_string(self):
        return ("Name: " +  str(self.name)+  " is at position "+  str(self.position) + " with " + str(self.get_num_soldiers()) + ' soldiers ')    
       
        

In [ ]:
class Country:
    
    """
    Class that houses the information for each individual country
    
    Attributes: 
    Money: Tells the amount of Money ~ GDP of the country. 
    Territory List: The territories the country possesses.
    Money Array: Money of each territory in Country
    
    Methods:
    add_territory/remove_territory: Adds/Remove territory from the country.
    add_GDP: Increases money of the country with time (because GDP increases)
    lose_money/add_money: This changes money depending on country winning/losing a war.
    
    """
    
    def __init__(self, name, territory_list, money, GDP_increase = 0.05):
        self.name = name
        self.deaths = 0
        self.territory_list = territory_list
    # Territory list will be initialzied as an attribute (List will come from terriotry class that Jonathan will make)
        self.money = money
    # Will be a number of money for the entire country in millions (again initialized once composition is used with Jonathan's territory class)
        self.GDP_increase = GDP_increase
    
    def add_territory(self, territory):
        self.territory_list.append(territory)
        self.money = self.money + territory.money
        
    def remove_territory(self, territory): 
        self.territory_list.remove(territory)
        self.money = self.money - territory.money
        self.deaths += territory.deaths
    
    def dist_money(self):
        # Once a territory wins/loses a battle the country needs to redistribute the money it has evenly over the number of territories
        money_per_territory = self.money / len(self.territory_list)
        for territory in self.territory_list:
            territory.money = money_per_territory
        
    def add_GDP(self):
        self.money += self.GDP_increase * self.money       

    def to_string(self):
        temp = ""
        for i in self.territory_list:
            temp += i.to_string()  + "\n"
            
        return temp
    
    def increase_soldiers(self):
        for terr in self.territory_list:
            terr.add_population()
            self.deaths += terr.add_soldiers()
            
    def add_money(self,amount):
        self.money += amount
        
    def remove_money(self,amount):
        self.money -= amount
        
    def lose_battle_money(self,territory,percent=0.1):
        self.money -= territory.money * percent

In [ ]:
"""
This is going to be our data for the simulation at least initially
"""

def take_in_data(file_name,country1_money,country2_money):

    territory_data = pd.read_csv(file_name)
    territory_data = territory_data.replace(0,"Country1")
    territory_data = territory_data.replace(1,"Country2")

    country_1_territory_list = []
    country_2_territory_list = []

    for i in range(len(territory_data)):
        if territory_data["country"][i] == "Country1":
            temp_Territory = Territory(territory_data["name"][i], [territory_data["longitude"][i],territory_data["latitude"][i] ],territory_data["population"][i])
            country_1_territory_list.append(temp_Territory)
        else:
            temp_Territory = Territory(territory_data["name"][i], [territory_data["longitude"][i],territory_data["latitude"][i] ],territory_data["population"][i])
            country_2_territory_list.append(temp_Territory)

    Country1 = Country('Red',country_1_territory_list,country1_money)
    Country2 = Country('Blue',country_2_territory_list,country1_money)
    
    return [Country1,Country2]

In [ ]:
def make_graph(Country1_list, Country2_list, draw = False):
    ''' Uses the two territories lists to make a color coded networkx graph and adds suitable edges
    to identify possible opposing territories and close neighbors. A territory can fight with
    another one if they are connected by an edge and are in different countries.
    '''
    Country1_dict = dict()
    Country1_pos = dict()
    all_pos = dict()
    for ter in Country1_list:
        Country1_dict[ter.name] = []
        Country1_pos[ter.name] = ter.position
        all_pos[ter.name] = ter.position
    Country2_dict = dict()
    Country2_pos = dict()
    for ter in Country2_list:
        Country2_dict[ter.name] = []
        Country2_pos[ter.name] = ter.position
        all_pos[ter.name] = ter.position
    
    if len(Country1_dict.keys()) != 0 and len(Country2_dict.keys()) != 0:
    
        G1 = nx.Graph(Country1_dict)
        G2 = nx.Graph(Country2_dict)
        G = nx.compose(G1,G2)
        #color code the two different countries
        node_color_dict = {}
        for node in G1.nodes():
            node_color_dict[node] = 'red'
        for node in G2.nodes():
            node_color_dict[node] = 'blue'

        node_colors = [node_color_dict[node] for node in G.nodes()]


        #add edge if two nodes are less than 20 units apart
        for node1, coords1 in all_pos.items():
            for node2, coords2 in all_pos.items():
                if node1 != node2:
                    distance = np.sqrt((coords1[0]-coords2[0])**2+(coords1[1]-coords2[1])**2)
                    if distance <= 20:
                        G.add_edge(node1, node2)


        #identify and add edge with the closest enemy territory
        closest_neighbors_Country1 = []
        for ter1,coords1 in Country1_pos.items():
            distances = []
            for ter2,coords2 in Country2_pos.items():
                distance = np.sqrt((coords1[0]-coords2[0])**2+(coords1[1]-coords2[1])**2)
                distances.append(distance)
            if len(distances) != 0:
                closest_dist = min(distances) 
                closest_neighbor_key = list(Country2_pos.keys())[distances.index(closest_dist)]
                closest_neighbors_Country1.append(closest_neighbor_key)

        closest_neighbors_Country2 = []
        for ter1,coords1 in Country2_pos.items():
            distances = []
            for ter2,coords2 in Country1_pos.items():
                distance = np.sqrt((coords1[0]-coords2[0])**2+(coords1[1]-coords2[1])**2)
                distances.append(distance)
            if len(distances) != 0:
                closest_dist = min(distances)
                closest_neighbor_key = list(Country1_pos.keys())[distances.index(closest_dist)]
                closest_neighbors_Country2.append(closest_neighbor_key)
        for i in range(len(Country1_pos.keys())):
            if len(Country1_pos.keys()) != 0:
                G.add_edge(list(Country1_pos.keys())[i],closest_neighbors_Country1[i])
            else:
                break
        for i in range(len(Country2_pos.keys())):
            if len(Country2_pos.keys()) != 0:
                G.add_edge(list(Country2_pos.keys())[i],closest_neighbors_Country2[i])
            else:
                break


        if draw == True:
            nx.draw(G, pos = all_pos, node_color=node_colors, with_labels = True)

        return G
    
    elif len(Country1_dict.keys()) != 0:
        G = nx.Graph(Country1_dict)
        nx.draw(G, pos = all_pos, node_color= 'red')
        return G
    else:
        G = nx.Graph(Country2_dict)
        nx.draw(G, pos = all_pos, node_color= 'blue')
        return G
        
        

In [ ]:

Country1, Country2 = take_in_data("Territory data.csv",1000,1000)

print("Country1 territories: ")
print(Country1.to_string())
print("\nCountry2 territories: ")
print(Country2.to_string())

In [ ]:
#Initial Base
make_graph(Country1.territory_list, Country2.territory_list, True)

In [ ]:
#WAR!!
def War(Country1,Country2):
    
    #Distribute Money to territories
    Country1.dist_money()
    Country2.dist_money()
    
    #Start the animation
    start = time.time()
    fig=plt.figure(figsize=(10,5))
    
    #each iteration is a year
    years = 0
    truth_value = True
    #loop until one country wins all the territories
#     while len(Country1.territory_list) != 0 and len(Country2.territory_list) != 0:
#     while Country1.money > 0 and Country2.money > 0:
    for count in range(800):
        #plot the Map
        G = make_graph(Country1.territory_list, Country2.territory_list,draw=True)
        clear_output(wait=True)
        display(fig)
        fig.clear()
        time.sleep(0.001)
        
        #randomizing the countries and territories to attack
        countries = [Country1, Country2]
        attacking_country = np.random.choice(countries)
        countries.remove(attacking_country)
        defending_country = countries[0]
        attacking_territory = np.random.choice(attacking_country.territory_list)
        possible_opponents = []
        for def_ter in defending_country.territory_list:
            if G.has_edge(attacking_territory.name, def_ter.name) == True:
                possible_opponents.append(def_ter)
        defending_territory = np.random.choice(possible_opponents)
        
        #one battle
        winning_territory = attacking_territory.attack(defending_territory)[0]
        losing_territory = attacking_territory.attack(defending_territory)[1]
        
        #adding and removing territories from the countries
        for country in [Country1,Country2]:
            if winning_territory in country.territory_list:
                winning_country = country
        if winning_country == Country1:
            Country1.add_territory(losing_territory)
            Country2.remove_territory(losing_territory)
            Country1.lose_battle_money(winning_territory)
        else:
            Country2.add_territory(losing_territory)
            Country1.remove_territory(losing_territory)
            Country2.lose_battle_money(winning_territory)
            
        #stop if one of the country wins
        if len(Country1.territory_list) == 0 or len(Country2.territory_list) == 0:
            make_graph(Country1.territory_list, Country2.territory_list,draw=True)
            clear_output(wait=True)
            display(fig)
            fig.clear()
            time.sleep(0.001)
            break
       

        #increase GDP of the country
        Country1.add_GDP()
        Country2.add_GDP()

        #Distribute Money
        Country1.dist_money()
        Country2.dist_money()

        #Increase Population and Soldiers
        Country1.increase_soldiers()
        Country2.increase_soldiers()
            
        years +=1



    end = time.time()
    print(years)  

In [ ]:
"""
The visualization of war of time. This is what is happening for each time step where 2 countries are fighting over territory
"""
War(Country1,Country2)

In [ ]:
def War_no_vis(Country1,Country2):
    
    #Distribute Money to territories
    Country1.dist_money()
    Country2.dist_money()
    
    #Start the animation
    
    #each iteration is a year
    years = 0
    truth_value = True
    #loop until one country wins all the territories
#     while len(Country1.territory_list) != 0 and len(Country2.territory_list) != 0:
#     while Country1.money > 0 and Country2.money > 0:
    for count in range(800):
        #plot the Map
        G = make_graph(Country1.territory_list, Country2.territory_list,draw=False)
        
        #randomizing the countries and territories to attack
        countries = [Country1, Country2]
        attacking_country = np.random.choice(countries)
        countries.remove(attacking_country)
        defending_country = countries[0]
        attacking_territory = np.random.choice(attacking_country.territory_list)
        possible_opponents = []
        for def_ter in defending_country.territory_list:
            if G.has_edge(attacking_territory.name, def_ter.name) == True:
                possible_opponents.append(def_ter)
        defending_territory = np.random.choice(possible_opponents)
        
        #one battle
        winning_territory = attacking_territory.attack(defending_territory)[0]
        losing_territory = attacking_territory.attack(defending_territory)[1]
        
        #adding and removing territories from the countries
        for country in [Country1,Country2]:
            if winning_territory in country.territory_list:
                winning_country = country
        if winning_country == Country1:
            Country1.add_territory(losing_territory)
            Country2.remove_territory(losing_territory)
            Country1.lose_battle_money(winning_territory)
        else:
            Country2.add_territory(losing_territory)
            Country1.remove_territory(losing_territory)
            Country2.lose_battle_money(winning_territory)
            
        #stop if one of the country wins
        if len(Country1.territory_list) == 0 or len(Country2.territory_list) == 0:
            break
            
        #increase GDP of the country
        Country1.add_GDP()
        Country2.add_GDP()
        
        #Distribute Money
        Country1.dist_money()
        Country2.dist_money()
        
        #Increase Population and Soldiers
        Country1.increase_soldiers()
        Country2.increase_soldiers()
        
        years +=1

    return years



In [ ]:
"""
This functions will run a number of simulations on the data that we provide it so that we can get an appoximate win % 
of the factors at play. We can then use the functions later on that are built off of these to customize the territories since 
we do not have text files with specific values

"""
def run_sims(num_sims,amount1,amount2):

    CountryA_death_array = np.zeros(num_sims)
    CountryB_death_array = np.zeros(num_sims)
    CountryA_money_array = np.zeros(num_sims)
    CountryB_money_array = np.zeros(num_sims)
    years_of_war = np.zeros(num_sims)
    winner_of_war = []

    for i in range(num_sims):
        Country1, Country2 = take_in_data("Territory data.csv",amount1,amount2)
        years_of_war[i] = War_no_vis(Country1,Country2)
        CountryA_death_array[i] = (Country1.deaths)
        CountryB_death_array[i] = (Country2.deaths)
        CountryA_money_array[i] = Country1.money
        CountryB_money_array[i] = Country2.money
        
        if len(Country1.territory_list) != 0 :
            winner_of_war.append(Country1.name)   
        else:
            winner_of_war.append(Country2.name)
            
    data={"A_deaths":CountryA_death_array,
          "B_deaths": CountryB_death_array,
          "A_money": CountryA_money_array,
          "B_money" : CountryB_money_array,
          "years_of_war" :years_of_war,
          "winner": winner_of_war}
        
    df = pd.DataFrame(data)
    return df

In [ ]:
#run simulations for 1000 times
df = run_sims(50,1000,1000)
#theres a chance that nobody wins so we remove this case
df = df[df["years_of_war"]!= 800]

In [ ]:
df.describe()

In [ ]:
#when there is constant money:
start_data= pd.read_csv("Territory data.csv")

print("Country 1")
print("started with a population of: ", sum(start_data["population"][start_data["country"]==0]))
print("wins :", round(len(df[df["winner"]=="Red"])/len(df) *100,2), "% of the time")
print("Mean number of casualties per war:" , np.mean(df['A_deaths']))

print("Country 2")
print("started with a population of: ", sum(start_data["population"][start_data["country"]==1]))
print("wins: ", round(len(df[df["winner"]=="Blue"])/len(df) *100,2), "% of the time")
print("Mean number of casualties per war:" , np.mean(df['B_deaths']))

In [ ]:
def run_sims_constant_soldiers(num_sims, amount1,amount2):

    CountryA_death_array = np.zeros(num_sims)
    CountryB_death_array = np.zeros(num_sims)
    CountryA_money_array = np.zeros(num_sims)
    CountryB_money_array = np.zeros(num_sims)
    years_of_war = np.zeros(num_sims)
    winner_of_war = []

    for i in range(num_sims):
        territory_data = pd.read_csv("Territory data.csv")
        territory_data = territory_data.replace(0,"Country1")
        territory_data = territory_data.replace(1,"Country2")

        country_1_territory_list = []
        country_2_territory_list = []

        for j in range(len(territory_data)):
            if territory_data["country"][j] == "Country1":
                temp_Territory = Territory(territory_data["name"][j], [territory_data["longitude"][j],territory_data["latitude"][j] ],150000)
                country_1_territory_list.append(temp_Territory)
            else:
                temp_Territory = Territory(territory_data["name"][j], [territory_data["longitude"][j],territory_data["latitude"][j] ],150000)
                country_2_territory_list.append(temp_Territory)

        Country1 = Country('Red',country_1_territory_list,amount1)
        Country2 = Country('Blue',country_2_territory_list,amount2)
        
        years_of_war[i] = War_no_vis(Country1,Country2)
        CountryA_death_array[i] = (Country1.deaths)
        CountryB_death_array[i] = (Country2.deaths)
        CountryA_money_array[i] = Country1.money
        CountryB_money_array[i] = Country2.money
        
        if len(Country1.territory_list) != 0 :
            winner_of_war.append(Country1.name)   
        else:
            winner_of_war.append(Country2.name)
            
    data={"A_deaths":CountryA_death_array,
          "B_deaths": CountryB_death_array,
          "A_money": CountryA_money_array,
          "B_money" : CountryB_money_array,
          "years_of_war" :years_of_war,
          "winner": winner_of_war}
        
    df = pd.DataFrame(data)
    return df

In [ ]:
df = run_sims_constant_soldiers(50,100000,20000)

In [ ]:
df = df[df["years_of_war"]!=800]

In [ ]:
df.describe()

In [ ]:
print("At the same initial populations")
print("Country 1")
print("started with $", 100000)
print("wins :", round(len(df[df["winner"]=="Red"])/len(df) *100,2), "% of the time")

print("Country 2")
print("started with $", 20000)
print("wins: ", round(len(df[df["winner"]=="Blue"])/len(df) *100,2), "% of the time")

In [ ]:
#what does better 15000 more soliders or  5x the money
df = run_sims(50,40000,20000)
df = df[df["years_of_war"]!=800]
df.describe()

In [ ]:
print("At the same initial populations")
print("Country 1")
print("started with $", 40000)
print("started with a population of: ", sum(start_data["population"][start_data["country"]==0]))
print("wins :", round(len(df[df["winner"]=="Red"])/len(df) *100,2), "% of the time")

print("Country 2")
print("started with $", 20000)
print("started with a population of: ", sum(start_data["population"][start_data["country"]==1]))
print("wins: ", round(len(df[df["winner"]=="Blue"])/len(df) *100,2), "% of the time")